# The simplest way to run the model

The main part is taken from [documentation](https://torcheeg.readthedocs.io/en/stable/auto_examples/examples_amigos_dgcnn.html)

In [1]:
import scipy 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
import torch.nn as nn
import math
import random

In [2]:
import torcheeg as teeg
from torcheeg.datasets.constants.emotion_recognition.dreamer import DREAMER_ADJACENCY_MATRIX, DREAMER_CHANNEL_LOCATION_DICT
from torcheeg.datasets.constants.emotion_recognition.seed import SEED_ADJACENCY_MATRIX, SEED_CHANNEL_LOCATION_DICT
from torcheeg.datasets.constants.emotion_recognition.seed_iv import SEED_IV_ADJACENCY_MATRIX, SEED_IV_CHANNEL_LOCATION_DICT
from torcheeg.datasets import DREAMERDataset, SEEDDataset, SEEDIVDataset
from torcheeg import transforms
from torcheeg.transforms.pyg import ToG

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(73)

In [4]:
DATA_DIR = os.path.join("..", "data")

# Arousal DREAMER part

In [5]:
dataset = DREAMERDataset(io_path=os.path.join(DATA_DIR, "dreamer"),
                         mat_path=os.path.join(DATA_DIR, 'DREAMER.mat'),
                         online_transform=transforms.ToTensor(),
                         offline_transform=transforms.Compose([
                             ToG(DREAMER_ADJACENCY_MATRIX)
                         ]),
                         label_transform=transforms.Compose([
                             transforms.Select('arousal'),
                             transforms.Binary(3.0)
                         ]),
                         # num_worker=4
                        )

dataset already exists at path ..\data\dreamer, reading from path...


In [6]:
from torcheeg.model_selection import KFoldPerSubject

SPLIT_PATH = os.path.join(DATA_DIR, 'dreamer_split')
k_fold = KFoldPerSubject(n_splits=5,
                         split_path=SPLIT_PATH,
                         shuffle=True)

In [ ]:
from torch.utils.data import DataLoader
from torcheeg.models import DGCNN

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(log_model="all")


for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    model = DGCNN(in_channels=128,
                  num_electrodes=14,
                  hid_channels=64,
                  num_classes=2)

    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=1e-4,
                                weight_decay=1e-4,
                                accelerator="gpu")
    trainer.fit(train_loader,
                val_loader,
                max_epochs=50,
                default_root_dir=os.path.join('.', "dgcnn_model", f'{i}'),
                callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
                logger=wandb_logger,
                enable_progress_bar=True,
                enable_model_summary=True,
                limit_val_batches=0.0)
    score = trainer.test(val_loader,
                         enable_progress_bar=True,
                         enable_model_summary=True)[0]
    print(f'Fold {i} test accuracy: {score["test_accuracy"]:.4f}')

2023-10-27 06:11:21,034 INFO MainThread pytorch_lightning.utilities.rank_zero GPU available: True (cuda), used: True
2023-10-27 06:11:21,036 INFO MainThread pytorch_lightning.utilities.rank_zero TPU available: False, using: 0 TPU cores
2023-10-27 06:11:21,037 INFO MainThread pytorch_lightning.utilities.rank_zero IPU available: False, using: 0 IPUs
2023-10-27 06:11:21,038 INFO MainThread pytorch_lightning.utilities.rank_zero HPU available: False, using: 0 HPUs
C:\Users\Georgy\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\loggers\wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
2023-10-27 06:11:21,851 INFO MainThread pytorch_lightning.accelerators.cuda LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2023-10-27 06:11:21,856 INFO MainThread pytorch_lightning.callbacks.model_summary 
  | Name          | Type        

Epoch 0: 100%|███████████████████| 187/187 [00:15<00:00, 12.23it/s, v_num=aujf, train_loss=0.692, train_accuracy=0.167]

2023-10-27 06:11:38,040 DEBUG MainThread fsspec.local open file: C:/Users/Georgy/Desktop/PhD/Skoltech/4term_T1/Neuroimaging_ML/PROJECT/STGATE/experiments/lightning_logs/5w91aujf/checkpoints/epoch=0-step=187.ckpt
2023-10-27 06:11:38,120 DEBUG MainThread urllib3.connectionpool Starting new HTTPS connection (1): api.wandb.ai:443



[Train] train_loss: 0.657 train_accuracy: 0.626 



2023-10-27 06:11:39,386 DEBUG MainThread urllib3.connectionpool https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


...


[Train] train_loss: 0.495 train_accuracy: 0.741 



2023-10-27 06:21:06,559 DEBUG MainThread urllib3.connectionpool Starting new HTTPS connection (1): api.wandb.ai:443
2023-10-27 06:21:08,369 DEBUG MainThread urllib3.connectionpool https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


Epoch 32: 100%|██████████████████| 187/187 [00:25<00:00,  7.31it/s, v_num=aujf, train_loss=0.542, train_accuracy=0.667]
[Train] train_loss: 0.490 train_accuracy: 0.730 



2023-10-27 06:21:34,008 DEBUG MainThread fsspec.local open file: C:/Users/Georgy/Desktop/PhD/Skoltech/4term_T1/Neuroimaging_ML/PROJECT/STGATE/experiments/lightning_logs/5w91aujf/checkpoints/epoch=32-step=6171.ckpt
2023-10-27 06:21:34,053 DEBUG MainThread urllib3.connectionpool Starting new HTTPS connection (1): api.wandb.ai:443
2023-10-27 06:21:35,863 DEBUG MainThread urllib3.connectionpool https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


Epoch 33:  15%|██▊                | 28/187 [00:03<00:19,  8.28it/s, v_num=aujf, train_loss=0.380, train_accuracy=0.750]

# Valence DREAMER part

In [5]:
dataset = DREAMERDataset(io_path=os.path.join(DATA_DIR, "dreamer"),
                         mat_path=os.path.join(DATA_DIR, 'DREAMER.mat'),
                         online_transform=transforms.ToTensor(),
                         offline_transform=transforms.Compose([
                             ToG(DREAMER_ADJACENCY_MATRIX)
                         ]),
                         label_transform=transforms.Compose([
                             transforms.Select('valence'),
                             transforms.Binary(3.0)
                         ]),
                         # num_worker=4
                        )

dataset already exists at path ..\data\dreamer, reading from path...


In [6]:
from torcheeg.model_selection import KFoldPerSubject

SPLIT_PATH = os.path.join(DATA_DIR, 'dreamer_split_valence')
k_fold = KFoldPerSubject(n_splits=5,
                         split_path=SPLIT_PATH,
                         shuffle=True)

In [7]:
from torch.utils.data import DataLoader
from torcheeg.models import DGCNN

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(log_model="all")


for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    model = DGCNN(in_channels=128,
                  num_electrodes=14,
                  hid_channels=64,
                  num_classes=2)

    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=1e-4,
                                weight_decay=1e-4,
                                accelerator="gpu")
    trainer.fit(train_loader,
                val_loader,
                max_epochs=50,
                default_root_dir=os.path.join('.', "dgcnn_model", f'{i}'),
                callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
                logger=wandb_logger,
                enable_progress_bar=True,
                enable_model_summary=True,
                limit_val_batches=0.0)
    score = trainer.test(val_loader,
                         enable_progress_bar=True,
                         enable_model_summary=True)[0]
    print(f'Fold {i} test accuracy: {score["test_accuracy"]:.4f}')

2023-10-27 07:19:06,696 DEBUG MainThread torchaudio._extension.utils Attempting to load FFmpeg version 6.
2023-10-27 07:19:08,624 DEBUG MainThread torchaudio._extension.utils Failed to load FFmpeg 6 extension.
Traceback (most recent call last):
  File "C:\Users\Georgy\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchaudio\_extension\utils.py", line 128, in _find_ffmpeg_extension
    return _find_versionsed_ffmpeg_extension(ffmpeg_ver)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Georgy\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchaudio\_extension\utils.py", line 113, in _find_versionsed_ffmpeg_extension
    _try_access_avutil(ffmpeg_ver)
  File "C:\Users\Georgy\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchaudio\_extension\utils.py", line 89, in _try_access_avutil
    torchaudio.lib._torchaudio.find_avutil(libavutil)
RuntimeError: error in LoadLibrary for avutil-58.dll. WinError 126: The specified module coul

2023-10-27 07:19:33,669 INFO MainThread pytorch_lightning.accelerators.cuda LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2023-10-27 07:19:33,677 INFO MainThread pytorch_lightning.callbacks.model_summary 
  | Name          | Type             | Params
---------------------------------------------------
0 | model         | DGCNN            | 74.4 K
1 | ce_fn         | CrossEntropyLoss | 0     
2 | train_loss    | MeanMetric       | 0     
3 | val_loss      | MeanMetric       | 0     
4 | test_loss     | MeanMetric       | 0     
5 | train_metrics | MetricCollection | 0     
6 | val_metrics   | MetricCollection | 0     
7 | test_metrics  | MetricCollection | 0     
---------------------------------------------------
74.4 K    Trainable params
0         Non-trainable params
74.4 K    Total params
0.297     Total estimated model params size (MB)
C:\Users\Georgy\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_datalo

Epoch 0: 100%|███████████████████| 187/187 [00:13<00:00, 13.55it/s, v_num=gby9, train_loss=0.738, train_accuracy=0.167]

2023-10-27 07:19:49,092 DEBUG MainThread fsspec.local open file: C:/Users/Georgy/Desktop/PhD/Skoltech/4term_T1/Neuroimaging_ML/PROJECT/STGATE/experiments/lightning_logs/lui8gby9/checkpoints/epoch=0-step=187.ckpt
2023-10-27 07:19:49,139 DEBUG MainThread urllib3.connectionpool Starting new HTTPS connection (1): api.wandb.ai:443



[Train] train_loss: 0.633 train_accuracy: 0.677 



...

C:\Users\Georgy\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
2023-10-27 07:37:28,379 INFO MainThread pytorch_lightning.utilities.rank_zero GPU available: True (cuda), used: True
2023-10-27 07:37:28,382 INFO MainThread pytorch_lightning.utilities.rank_zero TPU available: False, using: 0 TPU cores
2023-10-27 07:37:28,388 INFO MainThread pytorch_lightning.utilities.rank_zero IPU available: False, using: 0 IPUs
2023-10-27 07:37:28,392 INFO MainThread pytorch_lightning.utilities.rank_zero HPU available: False, using: 0 HPUs
2023-10-27 07:37:28,418 INFO MainThread pytorch_lightning.accelerators.cuda LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2023-10-27 07:37:28,428 DEBUG MainThread fsspec.local open file: C:/Users/Georgy/Desktop/PhD/Skoltech/4term_T1/Neuroimaging_ML/PROJECT/STGATE/experiments/lightning_logs/version_7/hparams.yaml
C:\Users\Georgy\AppData\Local\Programs\Python\Python31

Testing DataLoader 0:  91%|██████████████████████████████████████████████████████▉     | 43/47 [00:03<00:00, 12.29it/s]

2023-10-27 07:37:32,146 DEBUG MainThread fsspec.local open file: C:/Users/Georgy/Desktop/PhD/Skoltech/4term_T1/Neuroimaging_ML/PROJECT/STGATE/experiments/lightning_logs/version_7/hparams.yaml


Epoch 4:  99%|██████████████████▊| 185/187 [00:22<00:00,  8.39it/s, v_num=gby9, train_loss=0.423, train_accuracy=0.875]


TypeError: 'NoneType' object is not subscriptable